# Mass-Spring System Simulators

These simulations use Taichi. To run, first run the code cell below to import taichi

In [2]:
import taichi as ti

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, osx, python 3.9.7
[I 09/21/22 13:03:02.329 934842] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout


### Using GUI interface to control a ball drawn on the screen

In [27]:
gui = ti.GUI('Title', (640, 480))
pos = np.random.random((1,2))

while gui.running:
    gui.get_event()  # must be called before is_pressed
    if gui.is_pressed('a', ti.GUI.LEFT):
        pos[0,0]-=0.01
    elif gui.is_pressed('d', ti.GUI.RIGHT):
        pos[0,0]+=0.01
    elif gui.is_pressed('w', ti.GUI.UP):
        pos[0,1]+=0.01
    elif gui.is_pressed('s', ti.GUI.DOWN):
        pos[0,1]-=0.01
    gui.circles(pos, radius=5, palette=[0x068587],palette_indices=np.array([0]))
    gui.show()


In [10]:
import taichi as ti
ti.init()

N = 8
dt = 1e-5

x = ti.Vector.field(2, dtype=ti.f32, shape=N, needs_grad=True)  # particle positions
v = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle velocities
U = ti.field(dtype=ti.f32, shape=(), needs_grad=True)  # potential energy


@ti.kernel
def compute_U():
    for i, j in ti.ndrange(N, N):
        r = x[i] - x[j]
        # r.norm(1e-3) is equivalent to ti.sqrt(r.norm()**2 + 1e-3)
        # This is to prevent 1/0 error which can cause wrong derivative
        U[None] += -1 / r.norm(1e-3)  # U += -1 / |r|


@ti.kernel
def advance():
    for i in x:
        v[i] += dt * -x.grad[i]  # dv/dt = -dU/dx
    for i in x:
        x[i] += dt * v[i]  # dx/dt = v


def substep():
    with ti.ad.Tape(loss=U):
        # Kernel invocations in this scope will later contribute to partial derivatives of
        # U with respect to input variables such as x.
        compute_U(
        )  # The tape will automatically compute dU/dx and save the results in x.grad
    advance()


@ti.kernel
def init():
    for i in x:
        x[i] = [ti.random(), ti.random()]


init()
gui = ti.GUI('Autodiff gravity')
while gui.running:
    for i in range(50):
        substep()
    gui.circles(x.to_numpy(), radius=3)
    gui.show()

[Taichi] Starting on arch=x64


In [62]:
import math

gui = ti.GUI('Spring-Mass System', (640, 480))
#pos = np.array([[0.5, 0.5]])
xpos = 0.5
ypos = 0.5
mouse_y = 0 # just a placeholder

mass = gui.slider('Mass', 5, 15, step=1)
reset = gui.button('Reset')

mass.value = 10

while gui.running:
    # gui.get_event()  # must be called before is_pressed
    for e in gui.get_events(gui.PRESS):
        if e.key == 'a':
            xpos -= 0.01
            print("moving")
        elif e.key == gui.ESCAPE:
            gui.running = False
            print("ending sim")
            exit()
        elif e.key == gui.LMB:
            xpos = gui.get_cursor_pos()[0]
        elif e.key == reset:
            xpos = 0.5
    
    gui.circle((xpos,ypos), radius=mass.value)
    gui.show()

this happened
this happened
this happened
this happened
this happened
this happened
ending sim


Now beginning the spring process. 
This one mostly focuses on the widgets and using that to control the circle

In [9]:
import math
ti.init()

dt = 1e-5

# k = spring constant
# m = mass
# c = damping constant
def calc_angular(k, m, c): 
    return math.sqrt(k/m - math.pow(c/(2*m), 2))

gui = ti.GUI('Spring-Mass System', (640, 480))
#pos = np.array([[0.5, 0.5]])
xpos = 0.5
ypos = 0.5
mouse_y = 0 # just a placeholder

mass = gui.slider('Mass', 5, 15, step=1)
damping = gui.slider('Damping', 0, 100, step=5)
spring = gui.slider('Spring Constant', 0, 1000, step=20)
reset = gui.button('Reset')

mass.value = 5
damping.value = 2
spring.value = 5

while gui.running:
    for e in gui.get_events(gui.PRESS):
        if e.key == gui.ESCAPE:
            gui.running = False
            print("ending sim")
            exit()
        elif e.key == gui.LMB: 
            # now reset it and calculate sinusoid with a new initial amplitude
            xpos = gui.get_cursor_pos()[0]
            a_0 = abs(0.5-xpos)
        elif e.key == reset:
            xpos = 0.5
    
    gui.circle((xpos,ypos), radius=mass.value)
    gui.show()

[Taichi] Starting on arch=x64


"\nx = ti.Vector.field(2, dtype=ti.f32, shape=N, needs_grad=True)  # particle positions\nv = ti.Vector.field(2, dtype=ti.f32, shape=N)  # particle velocities\nU = ti.field(dtype=ti.f32, shape=(), needs_grad=True)  # potential energy\n\n\n@ti.kernel\ndef compute_U():\n    for i, j in ti.ndrange(N, N):\n        r = x[i] - x[j]\n        # r.norm(1e-3) is equivalent to ti.sqrt(r.norm()**2 + 1e-3)\n        # This is to prevent 1/0 error which can cause wrong derivative\n        U[None] += -1 / r.norm(1e-3)  # U += -1 / |r|\n\n\n@ti.kernel\ndef advance():\n    for i in x:\n        v[i] += dt * -x.grad[i]  # dv/dt = -dU/dx\n    for i in x:\n        x[i] += dt * v[i]  # dx/dt = v\n\n\ndef substep():\n    with ti.ad.Tape(loss=U):\n        # Kernel invocations in this scope will later contribute to partial derivatives of\n        # U with respect to input variables such as x.\n        compute_U(\n        )  # The tape will automatically compute dU/dx and save the results in x.grad\n    advance()

This is a simpler version of the one above (no sliders): 

### Known bugs

Not working correctly - I believe the ti.sin function only works with matrices, and was unable to find a suitable workaround, since I only needed it for one element

In [8]:

from math import sin
import taichi as ti

ti.init()

t = 0
dt = 1e-3

# Why do i need the substeps?
substeps = 10

x = ti.field(dtype=ti.f32, shape=(2,)) # x[0] = xpos, x[1] = ypos
mass = ti.field(dtype=ti.f32, shape=())
damping = ti.field(dtype=ti.f32, shape=())
spring = ti.field(dtype=ti.f32, shape=())
base = ti.field(dtype=ti.f32, shape=(2,))
a_0 = ti.field(dtype=ti.f32, shape=())

sqrt_field = ti.field(dtype=ti.f32, shape=())


# k = spring constant
# m = mass
# c = damping constant
@ti.func
def calc_angular(k, m, c): 
    return ti.sqrt(k/m - ti.pow(c/(2*m), 2))

@ti.kernel
def substep():
    # calculate the new x position
    # d = base[0] + a_0[None] * ti.exp(-damping[None] * t/ (2 * mass[None])) * ti.cos(calc_angular(spring[None], mass[None], damping[None]) * t)
    d = base[0] + a_0[None] * ti.sin(ti.sqrt(damping[None] / mass[None]) * t)
    # redo without substep
    
    # print(d)
    
    x[0] = d
    # calculate
    

def main():
    gui = ti.GUI('Sinusoidal Motion', (640, 480))
    
    t = 0
    dt = 1e-3
    
    x[0] = 0.5
    x[1] = 0.5
    base[0] = 0.5
    base[1] = 0.5
    # initialize values
    mass[None] = 0.5
    damping[None] = 2
    spring[None] = 10
    a_0[None] = 0
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key in [ti.GUI.ESCAPE, ti.GUI.EXIT]:
                exit()
            elif e.key == ti.GUI.LMB: #restarting sim from different spot
                # for now only simulating the sinusoidal motion in 1D
                x[0] = e.pos[0]
                a_0[None] = e.pos[0] - base[0]
                
                print("a_0 = ", a_0[None])
        
        # move stuff
        for step in range(substeps):
            substep()
            t += dt
        
        # Draw springs
        X = x.to_numpy()
        B = base.to_numpy()
        gui.line(begin=X, end=B, radius = 2, color=0x444444)
    
        gui.circle((x[0], x[1]), radius = 5)
        gui.show()
    
    
if __name__ == '__main__':
    main()



[Taichi] Starting on arch=x64
a_0 =  -0.47968751192092896
a_0 =  0.15156251192092896
a_0 =  -0.18906250596046448
a_0 =  0.13593751192092896
a_0 =  -0.1953125
a_0 =  0.1796875
a_0 =  0.004687488079071045


## Explicit Euler integration scheme

### 2 particle system with damping. 

Each particle's mass can be controlled individually using the keyboard

In [71]:
import taichi as ti

ti.init()

num_particles = 2
dt = 1e-3
substeps = 10

spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=num_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]
            
            
    # print(v[0], v[1])
    # print(f[0], f[1])
    # print("")
    

def main():
    gui = ti.GUI('Two Springs with damping', (640, 480))
    
    # initialize positions of particles
    x[0][0] = 0.7 # left x
    x[0][1] = 0.5 # left y
    x[1][0] = 0.3 # right x
    x[1][1] = 0.5 # right y
    # set initial velocities to 0
    v[0][0] = 0
    v[0][1] = 0
    v[1][0] = 0
    v[1][1] = 0
    
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(num_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'a': # lower left mass 
                mass[0] /= 1.1
            elif e.key == 'd': # increase left mass
                mass[0] *= 1.1
            elif e.key == 's': # lower right mass
                mass[1] /= 1.1
            elif e.key == 'w': # increase right mass
                mass[1] *= 1.1
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw spring
        X = x.to_numpy()
        gui.line(begin=X[0], end=X[1], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Press a to decrease the left mass, d to increase it',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press s to decrease the right mass, w to increase it',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'Left Mass {mass[0]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Right Mass {mass[1]:.1f}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()



[Taichi] Starting on arch=x64


### 3 particle system (fully connected) with damping

In [76]:
import taichi as ti

ti.init()

num_particles = 3
dt = 1e-3
substeps = 10

spring_k = ti.field(dtype=ti.f32, shape=())
drag_damping = ti.field(dtype=ti.f32, shape=())
mass = ti.field(dtype=ti.f32, shape=num_particles)

x = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=num_particles)
rest_length = ti.field(dtype=ti.f32, shape=())


@ti.kernel
def substep():
    
    # find force
    for i in range(num_particles):
        f[i] = [0,0] # reset force, since we're calculating it from scratch each time
        
        for j in range(num_particles):
            if i != j:
                x_ij = x[j] - x[i]
                d = x_ij.norm() # euclidean distance

                # spring force on i by j
                f[i] +=  spring_k[None] * x_ij / d * (d - rest_length[None])
                # damping force
                f[i] += drag_damping[None] * (v[j] - v[i]).dot(x_ij / d) * (x_ij / d)
        
    for i in range(num_particles):
        # update v
        v[i] = v[i] + dt * f[i] / mass[i]
        # update x
        x[i] = x[i] + dt * v[i]
            
            
    # print(v[0], v[1])
    # print(f[0], f[1])
    # print("")
    

def main():
    gui = ti.GUI('Two Springs with damping', (640, 480))
    
    # initialize positions of particles
    x[0][0] = 0.7 # left x
    x[0][1] = 0.5 # left y
    x[1][0] = 0.3 # right x
    x[1][1] = 0.5 # right y
    x[2][0] = 0.5 
    x[2][1] = 0.3 
    # set initial velocities to 0
    v[0][0] = 0
    v[0][1] = 0
    v[1][0] = 0
    v[1][1] = 0
    
    rest_length[None] = 0.3
    spring_k[None] = 10
    drag_damping[None] = 0.3
    for i in range(num_particles):
        mass[i] = 0.5
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == 'a': # lower left mass 
                mass[0] /= 1.1
            elif e.key == 'd': # increase left mass
                mass[0] *= 1.1
            elif e.key == 's': # lower right mass
                mass[1] /= 1.1
            elif e.key == 'w': # increase right mass
                mass[1] *= 1.1
            elif e.key == ti.GUI.UP:
                spring_k[None] *= 1.1
            elif e.key == ti.GUI.DOWN:
                spring_k[None] /= 1.1
            
        # Move stuff
        for step in range(substeps):
            substep()
        
        # Draw springs
        X = x.to_numpy()
        for i in range(num_particles):
            for j in range(num_particles):
                if i != j:
                    gui.line(begin=X[i], end=X[j], radius = 2, color = 0x444444)        
        # Draw particles
        for i in range(num_particles):
            gui.circle(pos=X[i], radius = 5)
        
        # show the current values
        gui.text(
            content='Press a to decrease the left mass, d to increase it',
            pos=(0,0.99),
            color=0xffffff)
        gui.text(
            content='Press s to decrease the right mass, w to increase it',
            pos=(0,0.95),
            color=0xffffff)
        gui.text(content=f'Left Mass {mass[0]:.1f}',
                 pos=(0, 0.90),
                 color=0xffffff)
        gui.text(content=f'Right Mass {mass[1]:.1f}',
                 pos=(0, 0.85),
                 color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()



[Taichi] Starting on arch=x64
